In [2]:
%pip install -Uq llama-index-llms-groq
%pip install -Uq llama-index
%pip install -Uq python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from llama_index.core.llms import ChatMessage
from llama_index.llms.groq import Groq
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

SYSTEM_PROMPT = '''

**Objective**:
Transform web-scraped markdown documents into a clear, comprehensive representation of essential content. The focus is on extracting all valuable information, ensuring that the main content of the page is fully captured and presented. While filtering out repetitive and irrelevant text, ensure that the content length is preserved and formatted into a well-structured markdown format. Ensure the output reflects the true content, context, and structure of the original webpage, maintaining the same language as the input document.

---

**Instructions for the System**:

1. **Initial Header**:  
   - Begin each transformed output with: `"# Response for <url>"`, inserting the URL found in the file. This acts as a consistent identifier for the source of the content.

2. **Complete Core Content Identification and Extraction**:  
   - Ensure that all main content from the raw markdown file is thoroughly examined and extracted, maintaining the full content length.
   - This includes detailed articles, main text bodies, descriptions, and any substantial information.
   - Guarantee that all content of informational value is included; nothing should be omitted that contributes to the page's main message and context.

3. **Link Evaluation and Inclusion**:
   - **Preserve** links only if they provide significant relevance, additional details, or direct relevance to the core content.
   - **Exclude** links commonly found in navigation menus, headers, footers, or site-wide elements like terms of service and privacy policy unless directly relevant to the page content.

4. **Information Structure and Formatting**:
   - Employ markdown syntax to organize the output effectively, using headings, subheadings, lists, and bullet points to replicate the hierarchical and categorical structure of the original content.
   - Ensure that content flows logically and contextually, maintaining the essence and message of the source material.
   - Highlight and retain any contextually important links using markdown `[link text](URL)` format.

5. **Content Integrity and Verbosity**:
   - Keep the extracted content faithful to its original form without paraphrasing, ensuring a detailed representation that captures all essential information and nuances.
   - Deliver a verbose output that preserves the full length of the original content, presenting an accurate and immediately usable document.

6. **Quality and Consistency**:
   - Ensure precision and uniformity across outputs for different input files, respecting the specificity and uniqueness of each webpage’s content.
   - Pay strict attention to the details that characterize and define the content’s context and purpose.

7. **Language Consistency**:
   - Always produce the output in the same language as the raw input document to maintain linguistic accuracy and coherence.

---

**Content Processing Instructions**:

- **Pattern-based Noise Removal**:
  - Remove elements that are typically navigational or promotional, such as:
    - Menu items (e.g., "Cursos", "Notícias", "Investigação", "Eventos", "Candidaturas").
    - Links unrelated to the main topic like "Página Inicial", "Office 365", or "Moodle".
    - Social media links (e.g., "Facebook", "Twitter", "Instagram").
    - Administrative or utility links and text items (e.g., "Login", "Subscribe", "Newsletter").
    - Repeated elements like footers and standard disclaimers.

- **Focus Extraction**:
  - Concentrate on sections that exhibit clearly defined information, such as structured headings in the raw markdown that correlate with substantive text.

- **Link Relevance Check**:
  - Retain links that significantly enhance the content, disregarding generic site links unless directly tied to the primary content focus.

- **Content Authenticity**:
  - Preserve the authenticity and factual representation of the webpage’s information, ensuring it aligns with the original page's context and message.

---

**Reminder**: This prompt is a guideline designed to cover a wide variety of content while consistently achieving the objective of extracting and structuring meaningful information. Always adapt according to the unique characteristics presented within each file, ensuring completeness, full capture of the main content with its length preserved, and maintaining the original language.

'''

APPEND_PROMPT = "RESPIRA FUNDO, REVÊ CUIDADOSAMENTE O CONTEÚDO ORIGINAL, E AGORA COMPLETA A VERSÃO FINAL DO CONTEÚDO FORMATADO. POR FAVOR, INCLUI TODA A INFORMAÇÃO RELEVANTE E FILTRA QUALQUER RUÍDO."

REVISOR_PROMPT_bk_0 = '''

Por favor, revê atentamente o teu texto formatado tendo em conta as seguintes instruções. Analisa o resultado anterior gerado pelo modelo juntamente com o prompt do sistema e o conteúdo bruto original. O objetivo é assegurar que o documento final formatado atende a todos os requisitos do sistema, capturando toda a informação relevante e mantendo a integridade do conteúdo.

1. **Compreensão Completa**: Verifica se todo o conteúdo principal do documento original foi capturado. Assegura que nenhuma informação relevante foi omitida e que o texto reflete fielmente o conteúdo original.

2. **Estrutura e Formatação**: Confirma que o texto está devidamente estruturado em markdown, utilizando cabeçalhos, subtítulos e listas para organizar a informação de forma lógica e categórica.

3. **Coerência de Língua**: Garante que o output é apresentado na mesma língua que o documento original, preservando a coerência linguística.

4. **Links e Contexto**: Certifica-te de que todos os links importantes e contextualmente relevantes foram incluídos, enquanto os links de navegação e não relevantes foram eliminados.

5. **Preservação do Conteúdo**: Assegura-te de que o comprimento do conteúdo não foi reduzido, mantendo toda a informação essencial e nuances do texto original.

Realiza as correções, adições ou ajustes necessários para alinhar o documento final com os requisitos do sistema. Quando terminares, o output deverá ser uma representação fiel, completa e bem estruturada do conteúdo original, pronto para utilização imediata sem necessidade de edição adicional.

'''


REVISOR_PROMPT = '''

Por favor, revê atentamente o teu texto formatado tendo em conta as seguintes instruções. Analisa o resultado anterior gerado pelo modelo juntamente com o SYSTEM PROMPT e o conteúdo bruto original. O objetivo é assegurar que o documento final formatado atende a todos os requisitos do sistema, capturando toda a informação relevante e mantendo a integridade do conteúdo.

1. **Compreensão Completa**: Verifica se todo o conteúdo principal do documento original foi capturado. Assegura que nenhuma informação relevante foi omitida e que o texto reflete fielmente o conteúdo original.

2. **Estrutura e Formatação**: Confirma que o texto está devidamente estruturado em markdown, utilizando cabeçalhos, subtítulos e listas para organizar a informação de forma lógica e categórica.

3. **Coerência de Língua**: Garante que o output é apresentado na mesma língua que o documento original, preservando a coerência linguística.

4. **Links e Contexto**: Certifica-te de que todos os links importantes e contextualmente relevantes foram incluídos, enquanto os links de navegação e não relevantes foram eliminados.

5. **Preservação do Conteúdo**: Assegura-te de que o comprimento do conteúdo não foi reduzido, mantendo toda a informação essencial e nuances do texto original.

Realiza as correções, adições ou ajustes necessários para alinhar o documento final com os requisitos do sistema. Quando terminares, o output deverá ser uma representação fiel, completa e bem estruturada do conteúdo original, pronto para utilização imediata sem necessidade de edição adicional.

**Nota super importante**: Ao responder, o modelo nunca deve incluir frases que são típicas de textos gerados por LLM que visam soar formais e estruturados, mas que muitas vezes parecem genéricas ou desnecessárias. Para evitar isso, é essencial usar uma linguagem direta e concisa, eliminando palavras ou frases de preenchimento que não acrescentam valor. O modelo deve priorizar a clareza e a concisão, apresentando apenas a informação essencial ou o resultado específico de forma direta. 

### O que descartar:

Evita incluir exemplos como a listagem abaixo, que contém informações de navegação, links de serviços e outras referências irrelevantes para o propósito da análise:

```markdown
## Universidade Lusófona
### Apresentação

A Universidade Lusófona é uma instituição de ensino superior que oferece uma variedade de cursos e programas acadêmicos.

### Serviços

- **Contacts**: https://www.ulusofona.pt/en/contacts
- **Password Change and Recovery**: https://secure.ensinolusofona.pt/alteracao_password/f?p=133:2
- **Help us to improve**: https://ulusofona.typeform.com/to/cipp2UFI
- **Lost and Found**: https://www.ulusofona.pt/en/lost-and-found

### Cursos

- **Bachelors**: https://www.ulusofona.pt/en/undergraduate
- **Masters**: https://www.ulusofona.pt/en/masters
- **PhD**: https://www.ulusofona.pt/en/phd
- **Post-graduation**: https://www.ulusofona.pt/en/post-graduation
- **All the courses**: https://www.ulusofona.pt/en/courses

### Documentos

- **Fees and Emoluments**: https://www.ulusofona.pt/en/documents?cat=5
- **Regulations and Orders**: https://www.ulusofona.pt/en/documents?cat=1
- **Forms**: https://www.ulusofona.pt/en/documents?cat=13
- **Reports**: https://www.ulusofona.pt/en/documents?cat=4
- **Validation of documents**: https://www.ulusofona.pt/en/validation-of-issued-documents

### Informação Acadêmica

- **Calendários Acadêmicos**: https://www.ulusofona.pt/en/calendars
- **Faculdades e Escolas**: https://www.ulusofona.pt/en/faculties-and-schools
- **Plano de Género e Diversidade**: https://www.ensinolusofona.pt/en/gender-and-diversity-plan
- **Propinas**: https://www.ulusofona.pt/en/fees
- **Motivos para Frequência**: https://razoes.ulusofona.pt/
- **Qualidade**: https://www.ulusofona.pt/en/qualidade
- **Sobre Nós**: https://www.ulusofona.pt/en/about
- **Visite-nos**: https://ulusofona.typeform.com/to/ypj6qk
- **Canal de Denúncias Internas**: https://www.ulusofona.pt/en/internal-reporting-channel

### Infraestruturas

- **Campus**: https://campus.ulusofona.pt/
- **Contactos**: https://www.ulusofona.pt/en/contacts
- **Entidade Promotora**: https://www.cofac.pt
- **Lusófona no Mundo**: https://www.ensinolusofona.pt/en/
- **Lusófona 360º**: https://vr360.ulusofona.pt/visitavirtual_EN/

### Professores

- **Avadoc**: https://www.ulusofona.pt/avadoc
- **Bem-vindos**: https://boasvindas.ulusofona.pt/
- **Carreira Docente**: https://www.ulusofona.pt/en/documents?q=Career
- **Diretório**: https://diretorio.ulusofona.pt/
- **Emprego Científico**: https://www.ulusofona.pt/en/open-positions/scientific-employment-and-research-grants
- **Portal do Docente**: https://secure.ensinolusofona.pt/ficha_docente/f?p=123:LOGIN_DESKTOP::::::
- **Portal do Colaborador**: https://colaborador.ensinolusofona.pt/mygiaf/Login.xhtml

### Investigação

- **Portal de Investigação**: https://research.ulusofona.pt/
- **ReCiL - Repositório Científico**: https://recil.ensinolusofona.pt/
- **Revistas Científicas**: https://revistas.ulusofona.pt/
- **Unidades de Investigação**: https://investigacao.ulusofona.pt/

### Recursos

- **Biblioteca**: https://biblioteca.ulusofona.pt/
- **Click - Portal de Aprendizagem**: https://www.ulusofona.pt/en/click
- **Documentos**: https://www.ulusofona.pt/documents
- **FAQ - Centro de Ajuda**: https://www.ulusofona.pt/en/faqs
- **Guia de Bem-vindo**: https://bemvindo.ulusofona.pt/
- **Logótipos e Identidade Gráfica**: https://www.ulusofona.pt/documentos?cat=3
- **Objetos Perdidos**: https://www.ulusofona.pt/en/lost-and-found
- **Regulamentos**: https://www.ulusofona.pt/en/documents?cat=1
- **Reshape**: https://secure.ensinolusofona.pt/reshape/
- **Serviços**: https://www.ulusofona.pt/en/services
- **Normas de Dissertações e Teses**: https://www.ulusofona.pt/media/normas-para-elaboracao-e-apresentacao-de-dissertacoes-e-teses.pdf

### Internacional

- **Estudantes Brasileiros**: https://www.ulusofona.pt/en/international-students/brazilian-students
- **Estudantes Internacionais**: https://www.ulusofona.pt/en/international-students
- **FILMEU - Universidade Europeia**: https://www.filmeu.eu/
- **Mobilidade Estudantil**: https://www.ulusofona.pt/en/mobility

### Estudantes

- **Calendário de Defesas de Dissertações**: https://www.ulusofona.pt/en/theses
- **Aplicativo Ensino Lusófona**: https://www.ulusofona.pt/en/services/mobile-app
- **Cartão de Estudante**: https://www.ulusofona.pt/en/news/students-card
- **Estágios**: https://eva.ulusofona.pt/
- **Estudantes**: https://www.ulusofona.pt/en/student
- **Necessidades Educativas Especiais**: https://www.ulusofona.pt/en/gaenee
- **Portal de Emprego**: https://eva.ulusofona.pt/portal-de-emprego-universia/
- **Orientador Estudantil**: https://www.ulusofona.pt/en/student-advisor
- **Bolsas de Estudo**: https://www.ulusofona.pt/en/acao-social-escolar
- **Vantagens e Benefícios**: https://www.ensinolusofona.pt/pt/vantagens

### Comunidade

- **Quartas na Lusófona**: https://www.ulusofona.pt/en/event/as-quartas-na-lusofona-23-24
- **Cinema Fernando Lopes**: https://www.ulusofona.pt/cinema-fernando-lopes
- **Construir Conhecimento**: https://www.ulusofona.pt/en/building-knowledge
- **Jornadas de Abertura**: https://www.ulusofona.pt/en/open-days
- **Escola Sénior**: https://escolasenior.ulusofona.pt/
- **Escola de Verão**: https://escolaverao.ulusofona.pt/
- **Hospital Veterinário - Marcação de Consultas**: https://www.ulusofona.pt/en/news/appointments-veterinary-hospital-
- **Lusófona Talks**: https://www.ulusofona.pt/en/lusofona-talks
- **Lusófona Verde**: https://www.ulusofona.pt/en/green-lusofona

### Media e Eventos

- **Crónicas**: https://www.ulusofona.pt/en/chronicles
- **Lições**: https://www.ulusofona.pt/en/lessons
- **Lusófona nos Media**: https://www.ulusofona.pt/en/lusofona-in-the-media
- **Minha História - Testemunhos**: https://www.ulusofona.pt/en/testimonials
- **Notícias**: https://www.ulusofona.pt/en/news
- **Podcast - Direta Sem Café**: https://www.ulusofona.pt/en/news/direta-sem-cafe-podcast-lusofona

```

'''

class GroqLLM:
    def __init__(self):
        self.llm = Groq(model="llama-3.1-70b-versatile", api_key=GROQ_API_KEY, temperature=0.1)

    def complete(self, prompt, context=None, system_prompt=SYSTEM_PROMPT):
        messages = [
            ChatMessage(role="system", content=system_prompt),
            ChatMessage(role="user", content=prompt)
        ]
        
        if context is not None:
            messages.append(ChatMessage(role="user", content=f"Context: {context}"))
        
        response = self.llm.chat(messages)
        return response.message.content

    def chat(self, messages):
        response = self.llm.chat(messages)
        return response.message.content

def process_file(file_path, llm):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Step 1: Initial processing
    initial_prompt = f"{content}\n\n{APPEND_PROMPT}"
    initial_response = llm.complete(initial_prompt)

    # Step 2: Revision
    revision_messages = [
        ChatMessage(role="system", content=SYSTEM_PROMPT),
        ChatMessage(role="user", content=initial_prompt),
        ChatMessage(role="assistant", content=initial_response),
        ChatMessage(role="user", content=REVISOR_PROMPT),
    ]
    final_response = llm.chat(revision_messages)

    return final_response #.message.content

def main():
    llm = GroqLLM()
    input_folder = 'output2'
    output_folder = 'output_clean_formatted'

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Process each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.md'):  # Assuming the files are markdown
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            print(f"Processing {filename}...")
            processed_content = process_file(input_path, llm)

            # Save the processed content
            with open(output_path, 'w', encoding='utf-8') as file:
                file.write(processed_content)

            print(f"Saved processed content to {output_path}")

if __name__ == "__main__":
    main()

Processing acao-social-escolar_desporto-e-bem-estar.md...


AttributeError: 'str' object has no attribute 'content'